# Extracción de los clips de la carpeta Demo

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import pandas as pd

## Obtenim els segons que duren cada clip

In [ ]:
import os
import cv2

carpeta_principal = 'lipread_mp4_demo/'

seconds_file = []
seconds_list = []
for entrada in os.listdir(carpeta_principal):
    ruta_absoluta = os.path.join(carpeta_principal, entrada)
    # Verifica si la entrada es una carpeta
    if os.path.isdir(ruta_absoluta):
        # Ruta de la carpeta que contiene las imágenes
        carpeta_1 = carpeta_principal + entrada
         # Itera sobre todos los archivos en la carpeta
        for subcarpeta in os.listdir(carpeta_1):
            # Itera sobre todas las carpetas test, train y val
            carpeta_2 = carpeta_1+'/'+subcarpeta
            for archivo in os.listdir(carpeta_2):
                # Verifica si el archivo es '.txt'
                if archivo.endswith('.txt'):
                    ruta_imagen_txt = os.path.join(carpeta_2, archivo)
                    # Leemos línea por línea el '.txt'
                    with open(ruta_imagen_txt, "r") as file:
                        lines = file.readlines()
                        # Accedemos a la línea número 5 y extraemos las letras de la posición 10 hasta la 13
                        seconds = lines[4][10:14]
                        seconds_file.append(float(seconds))
        seconds_list.append(seconds_file)
        seconds_file = []

## Funció per extreure els clips on diuen la paraula objectiu

In [ ]:
def frames_clip_extraction(video_path):
    import cv2
    
    # Abrir el video
    cap = cv2.VideoCapture(video_path)
    
    # Obtener la duración del video en segundos
    fps = cap.get(cv2.CAP_PROP_FPS)

    #Duración del vídeo real
    duration = 1.216
    
    # Obtener el tiempo de inicio y final del segmento que deseas extraer
    # start_time = (duration - 0.37) / 2  # Inicio del segmento en segundos
    start_time = round((duration - 0.37) / 2, 3)  # Redondear a 3 decimales
    # end_time = start_time + 0.37        # Fin del segmento en segundos
    end_time = round(start_time + 0.37, 3)  # Redondear a 3 decimales
    
    # Convertir el tiempo a frames
    start_frame = int(start_time * fps)
    end_frame = int(end_time * fps)
    
    # Almacenar los frames extraídos en una lista
    frames = []
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    while cap.isOpened() and cap.get(cv2.CAP_PROP_POS_FRAMES) <= end_frame:
        ret, frame = cap.read()
        if ret:
            frames.append(frame)
        else:
            break
    
    # Liberar recursos
    cap.release()

    return frames

In [ ]:
frames_clip = []
frames_clips_list = []

for entrada in os.listdir(carpeta_principal):
    ruta_absoluta = os.path.join(carpeta_principal, entrada)
    # Verifica si la entrada es una carpeta
    if os.path.isdir(ruta_absoluta):
        # Ruta de la carpeta que contiene las imágenes
        carpeta_1 = carpeta_principal + entrada
         # Itera sobre todos los archivos en la carpeta
        for subcarpeta in os.listdir(carpeta_1):
            # Itera sobre todas las carpetas test, train y val
            carpeta_2 = carpeta_1+'/'+subcarpeta
            for archivo in os.listdir(carpeta_2):
                # Verifica si el archivo es '.mp4'
                if archivo.endswith('.mp4'):
                    ruta_imagen_mp4 = os.path.join(carpeta_2, archivo)
                    # Extraemos los frames del clip de cada video
                    frames_clip.append(frames_clip_extraction(ruta_imagen_mp4))
        frames_clips_list.append(frames_clip)
        frames_clip = []

## Guardem els frames en una carpeta per al seu us

In [ ]:
len_frames_clips_list = []
for i in range(len(frames_clips_list)):
    print("Longitud de la carpeta frames_clips_list[{}]: {}".format(i, len(frames_clips_list[i])))
    len_frames_clips_list.append(len(frames_clips_list[i]))

In [ ]:
frames_lr_demo = []
for i in range(len(frames_clips_list)):
    for j in range(len(frames_clips_list[i])):
        frames_lr_demo.append(frames_clips_list[i][j])

In [ ]:
frames_lr_demo_np = np.array(frames_lr_demo)

In [ ]:
path = 'img_lipread_mp4_demo/'
cont = 0
for i in range(len(frames_lr_demo_np)):
    for j in range(len(frames_lr_demo_np[i])):
        cv2.imwrite(path + 'IMG_%06d.jpg' % cont, frames_lr_demo_np[i][j])
        cont = cont + 1